In [1]:
spark

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *  
#sc = SparkContext(appName = "workshop2019") 
#spark = SparkSession.Builder().getOrCreate() 

<a href='https://www.kaggle.com/kemical/kickstarter-projects'> Kickstarter-Project Kaggle data </a>  

In [3]:
!kaggle datasets download -d kemical/kickstarter-projects
#!chmod 600 /Users/chloe/.kaggle/kaggle.json  

kickstarter-projects.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!pwd

/Users/chloe/spark-2.4.0-bin-hadoop2.6


In [5]:
movieDf= sc.textFile('/Users/chloe/.kaggle/datasets/kemical/kickstarter-projects') 

In [6]:
movieDf.count() 

1010122

In [7]:
movieDf.take(10)  
#data.TakeSample(true, n, seed) 
#data.count() #to know how many lines in the db  

['ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real',
 '1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.00,2015-08-11 12:12:28,0.00,failed,0,GB,0.00,0.00,1533.95',
 '1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.00,2017-09-02 04:43:57,2421.00,failed,15,US,100.00,2421.00,30000.00',
 '1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.00,2013-01-12 00:20:50,220.00,failed,3,US,220.00,220.00,45000.00',
 '1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.00,2012-03-17 03:24:11,1.00,failed,1,US,1.00,1.00,5000.00',
 '1000011046,Community Film Project: The Art of Neighborhood Filmmaking,Film & Video,Film & Video,USD,2015-08-29,19500.00,2015-07-04 08:35:03,1283.00,canceled,14,US,1283.00,1283.00,19500.00',
 '1000014025,Monarch Espresso Bar,Restaurants,Fo

In [8]:
movieDf.persist() #.cache() 

/Users/chloe/.kaggle/datasets/kemical/kickstarter-projects MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [9]:
movieDf.map(lambda x: x[10]).filter(lambda x: x=='DE') 

PythonRDD[4] at RDD at PythonRDD.scala:53

In [10]:
df = spark.read.format("com.databricks.spark.csv").\
options(header="true", \
        inferschema="true").\
load("/Users/chloe/.kaggle/datasets/kemical/kickstarter-projects",header=True) 

df.show(100)
df.printSchema() 

+----------+--------------------+--------------------+-------------+--------+----------+----------+-------------------+-------------------+----------+----------+-------+-----------+----------------+-------------+
|        ID|                name|            category|main_category|currency|  deadline|      goal|           launched|            pledged|     state|   backers|country|usd pledged|usd_pledged_real|usd_goal_real|
+----------+--------------------+--------------------+-------------+--------+----------+----------+-------------------+-------------------+----------+----------+-------+-----------+----------------+-------------+
|1000002330|The Songs of Adel...|              Poetry|   Publishing|     GBP|2015-10-09|   1000.00|2015-08-11 12:12:28|               0.00|    failed|         0|     GB|       0.00|            0.00|      1533.95|
|1000003930|Greeting From Ear...|      Narrative Film| Film & Video|     USD|2017-11-01|  30000.00|2017-09-02 04:43:57|            2421.00|    faile

In [11]:
type(df) 

pyspark.sql.dataframe.DataFrame

In [12]:
df.filter(df['country']=='FR')\
.select(['ID','country','category', 'name']).show(5) 

+----------+-------+---------------+--------------------+
|        ID|country|       category|                name|
+----------+-------+---------------+--------------------+
| 100089599|     FR|        Apparel|CLOTHING LABEL WI...|
|1000916036|     FR|Performance Art|Conference Paradi...|
|1001216413|     FR| Product Design|Road Runner multi...|
|1002597730|     FR|    Video Games|     Dungeons Heroes|
| 100525355|     FR|    Video Games| The Stolen Crystals|
+----------+-------+---------------+--------------------+
only showing top 5 rows



In [13]:
df.filter(df['country']=='FR')\
.select(['country','category', 'name','usd pledged'])\
.orderBy(df['usd pledged'].desc(),df['name'])\
.show(10) 

+-------+--------------+--------------------+------------+
|country|      category|                name| usd pledged|
+-------+--------------+--------------------+------------+
|     FR|Product Design|Robust Domino - A...| 992.7245838|
|     FR|Product Design|Robust Domino - A...|      992.72|
|     FR|   Video Games|Pirates: Age of G...| 990.8334689|
|     FR|   Video Games|Pirates: Age of G...|      990.83|
|     FR|       Gadgets|Small GPS Tracker...|       99.30|
|     FR|       Gadgets|Small GPS Tracker...| 99.29881456|
|     FR|        Drinks|Khan Luxury Premi...|      989.26|
|     FR|        Drinks|Khan Luxury Premi...|989.25669165|
|     FR|Tabletop Games|StartUp, un jeu d...|      989.21|
|     FR|Tabletop Games|StartUp, un jeu d...|989.20617978|
+-------+--------------+--------------------+------------+
only showing top 10 rows



In [14]:
df.groupBy('name').count().show() 

+--------------------+-----+
|                name|count|
+--------------------+-----+
|WE ARE ONE: STREE...|    2|
|The Great Kentuck...|    2|
|Peak View Brewing...|    2|
|Little Boy Wonder...|    2|
|"Artlysium Presen...|    2|
|Gamerline Clothin...|    2|
|Disgruntled Decks...|    2|
|Thank You Letters...|    2|
|Z Day by Lena Pen...|    2|
|Fotomodel Lisa va...|    2|
|"Scorpius Dance T...|    2|
|     Uncommon Action|    2|
|Drift Plates X1 B...|    2|
|           Revoltoso|    2|
|quia non numquam ...|    2|
|          The Farmer|    4|
|Be a Turtle (and ...|    1|
|    Hodgello's Blood|    2|
|         Micro Match|    2|
|              Knits+|    2|
+--------------------+-----+
only showing top 20 rows



In [15]:
#Create a new DataFrame that contains
dfDe=df.filter(df["country"] == "DE")
dfDe.show() 

+----------+--------------------+----------------+-------------+--------+----------+---------+-------------------+---------+----------+-------+-------+-----------+----------------+-------------+
|        ID|                name|        category|main_category|currency|  deadline|     goal|           launched|  pledged|     state|backers|country|usd pledged|usd_pledged_real|usd_goal_real|
+----------+--------------------+----------------+-------------+--------+----------+---------+-------------------+---------+----------+-------+-------+-----------+----------------+-------------+
|1000338818|Diet! No thanks, ...|      Publishing|   Publishing|     EUR|2016-10-30| 10000.00|2016-08-31 18:13:01|    88.00|    failed|      9|     DE|      22.35|           97.62|     11092.99|
|1001188527|Design Your Own L...|         Fashion|      Fashion|     EUR|2016-06-14|  2000.00|2016-05-29 22:04:54|  2028.00|successful|      7|     DE|    2254.15|         2282.32|      2250.81|
|1001190550| Sanfilippo W

In [16]:
dfDe.groupBy('ID').count().sort(col('count'),asc="false").show() 

+----------+-----+
|        ID|count|
+----------+-----+
|1416611683|    1|
| 356478551|    1|
|2128714441|    1|
| 990214156|    1|
|1510921674|    1|
|1272340127|    1|
| 887993863|    1|
|1051661266|    1|
|1857625797|    1|
|1925949997|    1|
| 465092390|    1|
|  47356539|    1|
|1413998257|    1|
| 564146847|    1|
|1329907294|    1|
| 640636670|    1|
|1513017719|    1|
| 696699510|    1|
|1350211746|    1|
| 861644660|    1|
+----------+-----+
only showing top 20 rows



In [17]:
#use Pandas-like syntax
dfDE=df[df.country == 'DE']
dfDE.show()  

+----------+--------------------+----------------+-------------+--------+----------+---------+-------------------+---------+----------+-------+-------+-----------+----------------+-------------+
|        ID|                name|        category|main_category|currency|  deadline|     goal|           launched|  pledged|     state|backers|country|usd pledged|usd_pledged_real|usd_goal_real|
+----------+--------------------+----------------+-------------+--------+----------+---------+-------------------+---------+----------+-------+-------+-----------+----------------+-------------+
|1000338818|Diet! No thanks, ...|      Publishing|   Publishing|     EUR|2016-10-30| 10000.00|2016-08-31 18:13:01|    88.00|    failed|      9|     DE|      22.35|           97.62|     11092.99|
|1001188527|Design Your Own L...|         Fashion|      Fashion|     EUR|2016-06-14|  2000.00|2016-05-29 22:04:54|  2028.00|successful|      7|     DE|    2254.15|         2282.32|      2250.81|
|1001190550| Sanfilippo W

In [18]:
from pyspark.sql.functions import desc
#from pyspark.sql.functions import col 

df.groupBy('country').count().filter("`count`>2000").sort(desc("count")).show()  

+-------+------+
|country| count|
+-------+------+
|     US|547916|
|   null|304005|
|     GB| 61102|
|     CA| 26708|
|     AU| 14048|
|     DE|  6845|
|     NL|  5121|
|     FR|  4838|
|     IT|  4616|
|  N,0""|  3790|
|"N,""0"|  3783|
|     ES|  3643|
|     SE|  3025|
|     NZ|  2579|
+-------+------+



In [19]:
from pyspark.sql.functions import col 
df.filter(col('country')=='DE').filter(col('usd pledged')>100000) \
.select('ID','country', 'category','name','usd pledged').show()     

+----------+-------+----------------+--------------------+-----------+
|        ID|country|        category|                name|usd pledged|
+----------+-------+----------------+--------------------+-----------+
| 100635918|     DE|        Hardware|World's first wir...|  193622.51|
|1009154287|     DE|  Product Design|ZIPPELIN: An infl...|  148197.14|
|1021260036|     DE|        Hardware|YOUMO - Your Smar...|  197140.88|
|1136359599|     DE|        Hardware|The first plug & ...|  615908.67|
|  11439199|     DE|        Hardware|Kiën Light: Intel...|  133799.88|
|1208264559|     DE|  Product Design|MIITO – the susta...|  912907.01|
|1231508817|     DE|          Design|ComfyLight - The ...|  142252.29|
|1248088162|     DE|        Hardware|Noki: The smart d...|  430201.74|
| 126918586|     DE|Camera Equipment|solidLUUV: The Al...|  367300.46|
|1365570388|     DE|      Technology|Mellow – The Elec...|  345900.84|
|1392671263|     DE|      Technology|PACE - Mach dein ...|  378489.09|
|14032

In [20]:
df.filter(col('country')=='FR').filter(col('usd pledged')>100000) \
.select('ID','country', 'category','name','usd pledged').show()     

+----------+-------+----------------+--------------------+-----------+
|        ID|country|        category|                name|usd pledged|
+----------+-------+----------------+--------------------+-----------+
|   1069403|     FR|  Product Design|Plastic Bottle Cu...|  396262.03|
|1074092371|     FR|        Hardware|SilentKeys: A Key...|  123135.70|
|  11436524|     FR|     Accessories|Watches / Vintage...|  216038.00|
|1182528526|     FR|  Tabletop Games|Kharnage... Hell ...|  189200.77|
|1186035417|     FR|  Tabletop Games|Heroes of Normand...|  225081.85|
|1187599692|     FR|Camera Equipment|SteadXP - The Fut...|  565333.82|
|  12134418|     FR|          Design|New AMIGA 1200 Ca...|  176145.80|
|1222247098|     FR|  Tabletop Games|Fleet Commander -...|  238272.05|
|1226388618|     FR|        Hardware|Mixfader - the wo...|  121378.48|
|1228706685|     FR|  Product Design|BALTIC Watches: N...|  315006.32|
|1290850818|     FR|  Product Design|Amazing furniture...|  126006.89|
|12970

In [21]:
df.select(df['ID'],df['country']=='DE', \
          df['category'],upper(df['name']),\
          (df['usd pledged']>200000).alias('big pledged'))\
.show(5)   

+----------+--------------+--------------+--------------------+-----------+
|        ID|(country = DE)|      category|         upper(name)|big pledged|
+----------+--------------+--------------+--------------------+-----------+
|1000002330|         false|        Poetry|THE SONGS OF ADEL...|      false|
|1000003930|         false|Narrative Film|GREETING FROM EAR...|      false|
|1000004038|         false|Narrative Film|      WHERE IS HANK?|      false|
|1000007540|         false|         Music|TOSHICAPITAL REKO...|      false|
|1000011046|         false|  Film & Video|COMMUNITY FILM PR...|      false|
+----------+--------------+--------------+--------------------+-----------+
only showing top 5 rows



In [22]:
from pyspark.sql.functions import udf
upper = udf(lambda s: s.upper())
df.select(upper(df['name'])).show(5)

+--------------------+
|      <lambda>(name)|
+--------------------+
|THE SONGS OF ADEL...|
|GREETING FROM EAR...|
|      WHERE IS HANK?|
|TOSHICAPITAL REKO...|
|COMMUNITY FILM PR...|
+--------------------+
only showing top 5 rows



In [23]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

columns = ['id', 'city', 'country']
vals = [(1, 'berlin', 'germany'),
        (2, 'shanghai', 'china'),
        (3,'new york', 'usa'),
        (4, 'london', 'gb')
]

#dfCity= sqlContext.createDataFrame(vals, columns) 
dfCity= spark.createDataFrame(vals, columns) 
dfCity.show() 
#use sql func? 

+---+--------+-------+
| id|    city|country|
+---+--------+-------+
|  1|  berlin|germany|
|  2|shanghai|  china|
|  3|new york|    usa|
|  4|  london|     gb|
+---+--------+-------+



In [24]:
dfCity.printSchema() 

root
 |-- id: long (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)



In [28]:
#dataframe and sql can be mixed 
from pyspark.sql import SQLContext
sqlContext=SQLContext(sc) 
df.createTempView("kickstarterView")
sqlContext.sql("SELECT * FROM kickstarterView").show()  

+----------+--------------------+--------------+-------------+--------+----------+---------+-------------------+--------+----------+-------+-------+-----------+----------------+-------------+
|        ID|                name|      category|main_category|currency|  deadline|     goal|           launched| pledged|     state|backers|country|usd pledged|usd_pledged_real|usd_goal_real|
+----------+--------------------+--------------+-------------+--------+----------+---------+-------------------+--------+----------+-------+-------+-----------+----------------+-------------+
|1000002330|The Songs of Adel...|        Poetry|   Publishing|     GBP|2015-10-09|  1000.00|2015-08-11 12:12:28|    0.00|    failed|      0|     GB|       0.00|            0.00|      1533.95|
|1000003930|Greeting From Ear...|Narrative Film| Film & Video|     USD|2017-11-01| 30000.00|2017-09-02 04:43:57| 2421.00|    failed|     15|     US|     100.00|         2421.00|     30000.00|
|1000004038|      Where is Hank?|Narrati

In [40]:
sqlContext.sql('''
SELECT ID, name, category, `usd pledged`
    FROM kickstarterView 
    where country= 'DE'
    ORDER BY name 
    limit 10 
''').show()    

+----------+--------------------+-----------+-------------+
|        ID|                name|   category|  usd pledged|
+----------+--------------------+-----------+-------------+
|1812477194|"""A Piece for Yo...|        Art|       538.54|
|1812477194|"""A Piece for Yo...|        Art|   538.538983|
|1802330047|"""ARTH?LL"" is t...|Mixed Media|            0|
|1802330047|"""ARTH?LL"" is t...|Mixed Media|         0.00|
|2042114874|"""All is forgive...|      Music|            0|
|2042114874|"""All is forgive...|      Music|         0.00|
|1843275794|"""BERLIN black &...| Photobooks|1915.35455511|
|1843275794|"""BERLIN black &...| Photobooks|      1915.35|
|1252454045|"""BERLIN schwarz...|Photography|      2304.41|
|1252454045|"""BERLIN schwarz...|Photography| 2304.4111506|
+----------+--------------------+-----------+-------------+

